# Nettoyage des données de Formula 1

## 1. Objectifs du Notebook
Ce notebook a pour objectif principal de préparer les données de Formula 1 en vue d’analyses approfondies. Le dataset couvre une période allant de 1950 à 2024, incluant divers aspects tels que les circuits, les pilotes, les classements, les résultats, et bien plus. Pour garantir la fiabilité des analyses, plusieurs tâches de nettoyage des données seront effectuées :

#### Validation des types de données :

- Vérifier que chaque colonne utilise un type de données approprié (numérique, catégoriel, etc.).


#### Identification et gestion des valeurs manquantes :

- Repérer les colonnes ou lignes contenant des données manquantes et appliquer des stratégies adaptées (suppression ou imputation).
  
#### Traitement des doublons :

- Vérifier l’existence de doublons dans les fichiers et les éliminer pour éviter les biais dans les analyses.

#### Gestion des incohérences :

- Identifier et corriger les éventuelles incohérences dans les données (par exemple, une valeur qui sort du bon intervalle).

#### Uniformisation des formats :

- Harmoniser les formats des dates, heures, noms, et autres champs pour assurer la cohérence des données.

Le but final est de produire un jeu de données propre, structuré et prêt pour des analyses exploratoires ou la construction de modèles de machine learning.

## 2. Importation des packages

In [1]:
import pandas as pd
import seaborn as sns

## 3. Chargement des données

In [24]:
# Chargement des données 
meteo = pd.read_csv('Data/Weather(2018 - 2024).csv')
courses = pd.read_csv('Data/races.csv')
pilotes = pd.read_csv('Data/drivers.csv')
resultat = pd.read_csv('Data/results.csv')
laps_times = pd.read_csv('Data/lap_times.csv')

# Afficher les dimensions de chaque dataset
print("Dimensions de 'meteo' :", meteo.shape)
print("Dimensions de 'courses' :", courses.shape)
print("Dimensions de 'pilotes' :", pilotes.shape)
print("Dimensions de 'resultats' :", resultat.shape)
print("Dimensions de 'laps_times' :", laps_times.shape)

Dimensions de 'meteo' : (21859, 10)
Dimensions de 'courses' : (1125, 18)
Dimensions de 'pilotes' : (859, 9)
Dimensions de 'resultats' : (26519, 18)
Dimensions de 'laps_times' : (575029, 6)


## 4. Nettoyage de la base de données Météo

In [3]:
## Affichages des 5 premières lignes
meteo.head(5)

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed,Round Number,Year
0,0 days 00:00:57.060000,24.1,36.2,997.1,False,38.2,294,3.0,1,2018
1,0 days 00:01:57.078000,24.0,36.3,997.1,False,38.6,273,1.4,1,2018
2,0 days 00:02:57.090000,24.0,36.3,997.1,False,38.6,273,1.4,1,2018
3,0 days 00:03:57.106000,23.9,37.2,997.0,False,38.7,287,2.3,1,2018
4,0 days 00:04:57.121000,24.2,35.8,997.1,False,38.7,309,3.5,1,2018


In [4]:
## Vérification des types des données
meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21859 entries, 0 to 21858
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Time           21859 non-null  object 
 1   AirTemp        21859 non-null  float64
 2   Humidity       21859 non-null  float64
 3   Pressure       21859 non-null  float64
 4   Rainfall       21859 non-null  bool   
 5   TrackTemp      21859 non-null  float64
 6   WindDirection  21859 non-null  int64  
 7   WindSpeed      21859 non-null  float64
 8   Round Number   21859 non-null  int64  
 9   Year           21859 non-null  int64  
dtypes: bool(1), float64(5), int64(3), object(1)
memory usage: 1.5+ MB


### Vérification des données manquantes

In [5]:
# Fonction qui vérifie s'il y a des valeurs manquantes dans un dataset
def check_missing_values(df, dataset_name="Dataset"):
    if df.isnull().sum().sum() == 0 :
        print(f'La base de données {dataset_name} ne contient pas des valeurs manquantes')
    else:
        print(f'La base de données {dataset_name} contient {df.isnull().sum().sum()} valeur(s) manquante(s)')

check_missing_values(meteo, 'Météo')

La base de données Météo ne contient pas des valeurs manquantes


### Vérification des doublons

In [6]:
# Fonction qui vérifie s'il y a des doublons dans un dataset
def check_duplicates(df, dataset_name="Dataset"):
    duplicates = df.duplicated().sum()  # Compter le nombre de doublons
    if duplicates > 0:
        print(f"{dataset_name} contient {duplicates} doublon(s).")
    else:
        print(f"{dataset_name} ne contient pas de doublons.")

check_duplicates(meteo, 'Météo')

Météo contient 481 doublon(s).


In [7]:
# Suppression des doublons
def remove_duplicates(df):
    return df.drop_duplicates()

meteo_cleaned = remove_duplicates(meteo)

In [8]:
# Vérification des la suppression des doublons
check_duplicates(meteo_cleaned, 'Météo')

Météo ne contient pas de doublons.


### Vérification des intervalles pour les colonnes numériques

In [9]:
meteo_cleaned.describe()

,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Round Number,Year
count,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000
mean,23.471181,54.709664,986.793068,35.224815,173.404107,1.637702,11.524652,2021.365656
std,4.988278,17.379909,50.957704,9.251853,104.043864,1.148441,6.302429,1.964110
min,8.900000,5.000000,778.500000,13.800000,0.000000,0.000000,1.000000,2018.000000
25%,20.000000,42.000000,986.200000,29.000000,95.000000,0.800000,6.000000,2020.000000
50%,23.200000,55.000000,1006.600000,34.800000,170.000000,1.400000,12.000000,2022.000000
75%,27.300000,65.775000,1013.700000,42.200000,264.000000,2.200000,17.000000,2023.000000
max,37.200000,97.500000,1023.500000,67.000000,359.000000,10.100000,24.000000,2024.000000


Toutes les variables du dataset sont comprises dans des intervalles réalistes, à l'exception de la variable Pressure. 

Cette dernière présente des valeurs comprises entre 778 et 1023, ce qui est incohérent avec les observations météorologiques réelles. En effet, la pression atmosphérique la plus faible jamais enregistrée est de 879 hPa (source : https://alarmemeteo.ch/blog/la-pression-atmospherique-la-declencheuse-des-vents-et-des-tempetes.html). 

Par conséquent, dans la suite de ce projet, seules les pressions atmosphériques égales ou supérieures à 900 hPa seront conservées pour garantir une analyse cohérente et fiable.

In [10]:
# Proportion des meteo qui ont une pression alterieur inferieur à 950
(meteo['Pressure'] < 900).sum() / len(meteo) * 100

4.336886408344389

Nous constatons que Environ 4% des courses ont une pression atmospherique inférieur à 900 hPa

In [11]:
# Suppression des valeurs de pression atmosphérique < 950
meteo_cleaned = meteo_cleaned[meteo_cleaned['Pressure'] >= 950]

# Afficher les 5 premières lignes pour vérifier
meteo_cleaned.describe()

,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Round Number,Year
count,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000,18220.000000
mean,23.661465,55.770692,1003.965368,34.556811,171.775521,1.672849,10.689133,2021.338749
std,5.085639,17.062601,14.715938,8.795537,105.520129,1.186057,6.047923,1.952609
min,9.200000,5.000000,959.900000,13.800000,0.000000,0.000000,1.000000,2018.000000
25%,19.700000,44.000000,995.900000,28.700000,93.000000,0.800000,5.000000,2020.000000
50%,24.000000,56.000000,1008.500000,34.000000,167.000000,1.400000,11.000000,2022.000000
75%,27.700000,66.200000,1014.900000,41.000000,263.000000,2.200000,15.000000,2023.000000
max,37.200000,97.500000,1023.500000,67.000000,359.000000,10.100000,24.000000,2024.000000


### Enregistrement des données nettoyés dans un fichier csv

In [22]:
meteo_cleaned.to_csv('Cleaned_Data/meteo_cleaned.csv', index=False)

print("Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/meteo_cleaned.csv'.")

Les données nettoyées ont été enregistrées dans le fichier 'Cleaned_Data/meteo_cleaned.csv'.
